<a href="https://colab.research.google.com/github/annasajkh/AutoEncoder/blob/main/Conv_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["KAGGLE_CONFIG_DIR"] = "/content/drive/MyDrive/kaggle"

!kaggle datasets download -d greatgamedota/ffhq-face-data-set

In [ ]:
!unzip ffhq-face-data-set.zip

In [ ]:
import glob
from PIL import Image
import numpy as np

imgs = []

count = 0

for file in glob.glob("thumbnails128x128/*"):
  imgs.append(np.array(Image.open(file).resize((64,64))) / 255)
  count += 1

  if count == 20_000:
    break

np.save("face_dataset.npy", np.array(imgs))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp drive/MyDrive/face_dataset.npy /content/

In [ ]:
from torch.nn.modules.linear import Linear
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
import numpy as np

class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()

    self.encoder = nn.Sequential(
      nn.Conv2d(3, 120, kernel_size=3),
      nn.LeakyReLU(True),
      nn.MaxPool2d(2, 2),

      nn.Conv2d(120, 160, kernel_size=5),
      nn.LeakyReLU(True),
      nn.MaxPool2d(2, 2),

      nn.Conv2d(160, 200, kernel_size=3),
      nn.LeakyReLU(True),
      nn.MaxPool2d(2, 2),

      nn.Flatten()
    )

    self.fc1 = nn.Sequential(
      nn.Linear(5000, 500),
      nn.Tanh()
    )

    self.fc2 = nn.Sequential(
      nn.Linear(500, 5000),
      nn.Tanh()
    )


    self.decoder = nn.Sequential(      
      nn.ConvTranspose2d(200, 160, kernel_size=3, stride=2),
      nn.LeakyReLU(True),

      nn.ConvTranspose2d(160, 120, kernel_size=2, stride=3),
      nn.LeakyReLU(True),

      nn.ConvTranspose2d(120, 3, kernel_size=2, stride=2),
      nn.Sigmoid()
    )

  def forward(self, x):
    x = self.encode(x)
    x = self.decode(x)
    return x
  
  def encode(self, x):
    x = self.encoder(x)
    x = self.fc1(x)
    return x

  def decode(self, x):
    x = self.fc2(x)
    x = x.view(x.shape[0], 200, 5, 5)
    x = self.decoder(x)
    return x


model = AutoEncoder()
optimizer = Adam(model.parameters(), lr=0.0001)
loss_function = nn.BCELoss()

if torch.cuda.is_available():
    model = model.cuda()
    loss_function = loss_function.cuda()


In [ ]:
from tqdm import tqdm

epoch = 10_000

dataset = torch.from_numpy(np.load("face_dataset.npy")).float()

if torch.cuda.is_available():
  dataset = dataset.cuda()

dataset = dataset.view(len(dataset), 3, 64, 64)

batch_size = 32

In [ ]:
for _ in tqdm(range(epoch)):
  for i in range(1, int(len(dataset) / batch_size)):
    data = dataset[(i-1)*batch_size:i*batch_size]
    
    optimizer.zero_grad()
    output = model(data)

    loss = loss_function(output, data)
    loss.backward()
    optimizer.step()